In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve, minimize
from scipy.optimize import minimize, LinearConstraint
import warnings

In [2]:
# Define sector-specific parameters

δ=0.05
σ=2.2
γ=0.106

psi_vec = {
    "Energy": {"DAC": 1, "China": 0.84},
    "Health": {"DAC": 1, "China": 0.78},
    "Education": {"DAC": 1, "China": 0.99},
    "Water Supply & Sanitation": {"DAC": 1, "China": 1},
    "Government & Civil Society": {"DAC": 1, "China": 1},
    "General Budget Support": {"DAC": 1, "China": 0.73},
    "Other Social Infrastructure & Services": {"DAC": 1, "China": 1},
    "Agriculture, Forestry, Fishing": {"DAC": 1, "China": 0.83},
    "Industry, Mining, Construction": {"DAC": 1, "China": 0.42},
    "Transport & Storage": {"DAC": 1, "China": 1},
    "Communications": {"DAC": 1, "China": 0.65},
    "General Environment Protection": {"DAC": 1, "China": 1},
    "General Economic, Commercial, Labor Affairs": {"DAC": 1, "China": 1}
}

R_vec = {
    "Agriculture, Forestry, Fishing": {"DAC": 1.009, "China": 1.025},
    "Industry, Mining, Construction": {"DAC": 1.011, "China": 1.039},
    "Transport & Storage": {"DAC": 1.010, "China": 1.033},
    "Energy": {"DAC": 1.013, "China": 1.040},
    "Communications": {"DAC": 1.009, "China": 1.031},
    "Health": {"DAC": 1.009, "China": 1.023},
    "Education": {"DAC": 1.009, "China": 1.026},
    "General Environment Protection": {"DAC": 1.013, "China": 1.030},
    "Water Supply & Sanitation": {"DAC": 1.011, "China": 1.027},
    "Government & Civil Society": {"DAC": 1.010, "China": 1.018},
    "General Budget Support": {"DAC": 1.011, "China": 1.045},
    "General Economic, Commercial, Labor Affairs": {"DAC": 1.015, "China": 1.038},
    "Other Social Infrastructure & Services": {"DAC": 1.012, "China": 1.020}
}

gamma_vec = {
    "Agriculture, Forestry, Fishing":0.0119 ,
    "Industry, Mining, Construction":0.0029 ,
    "Transport & Storage":0.0573 ,
    "Energy":0.0053 ,
    "Communications":0.0004 ,
    "Health":0.1429 ,
    "Education":0.1297 ,
    "General Environment Protection":0.0169 ,
    "Water Supply & Sanitation":0.0196 ,
    "Government & Civil Society":0.0449 ,
    "General Budget Support":0.1434 ,
    "General Economic, Commercial, Labor Affairs":0.0253 ,
    "Other Social Infrastructure & Services": 0.3613,
    "Defense": 0.0382
}

In [3]:
data_prep2 = pd.read_stata('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\GMM_prep2.dta')
data_ids = pd.read_stata('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\IDS_GMM.dta')
data_xi = data_prep2[['iso_code','xi']]
data_chi = data_prep2[['iso_code','chi_bad']]

In [4]:
xi_vec = data_xi.set_index('iso_code')['xi'].to_dict()
chi_vec = data_chi.set_index('iso_code')['chi_bad'].to_dict()

In [5]:
data_prep2['sec_fin_bad'].unique()

array(['Ctilde_Gtilde', 'Ctilde_Dtilde_Gtilde', 'Dtilde_Ctilde_Gtilde',
       'Dtilde_Gtilde', 'Gtilde'], dtype=object)

In [6]:
def define_parameter_dictionary(iso_code):
    """
    Define a nested parameter dictionary for a given country identified by its ISO code.
    The dictionary is structured by sectors, with each sector containing its related parameters.
    
    Parameters:
        iso_code (str): The ISO code for the country.
        data_prep2 (DataFrame): The dataframe containing sector and financial data.
    
    Returns:
        dict: A nested dictionary of parameters required for the GMM estimation, initialized to None or default values.
    """
    # Start with an empty dictionary
    parameter_dict = {}

    # Check if the country is in the data
    if iso_code not in data_prep2['iso_code'].values:
        return parameter_dict  # Return the empty dictionary if no sectors are found

    # Get all sectors for the given country
    sectors = data_prep2[data_prep2['iso_code'] == iso_code]['sector'].unique()
    
    # Pre-fetch sector financial data to reduce DataFrame access in the loop
    sector_data = data_prep2[data_prep2['iso_code'] == iso_code][['sector', 'sec_fin']].drop_duplicates()

    # Loop through each sector to determine the parameters to include
    for sector in sectors:
        # Initialize a sub-dictionary for each sector
        parameter_dict[sector] = {}
        
        sec_fin = sector_data[sector_data['sector'] == sector]['sec_fin'].values[0]
        
        # Define which parameters to initialize based on 'sec_fin'
        if sec_fin in ('G', 'Gtilde'):
            parameter_dict[sector]['f_g'] = None
        elif sec_fin in ('D_G', 'Dtilde_Gtilde'):
            parameter_dict[sector]['f_g'] = None
            parameter_dict[sector]['f_d'] = None
        elif sec_fin in ('Ctilde_Gtilde'):
            parameter_dict[sector]['f_g'] = None
            parameter_dict[sector]['f_c'] = None
        elif sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde', 'Ctilde_Dtilde_Gtilde'):
            parameter_dict[sector]['f_g'] = None
            parameter_dict[sector]['f_d'] = None
            parameter_dict[sector]['f_c'] = None
    
    return parameter_dict

In [7]:
def calcul_fc_G_good(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = 0
    
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_good'].values[0] 
        
        if sec_fin in ('G', 'Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            f_g = empirical_g / common_g
        
    return f_g 

def calcul_fc_DG_good(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = 0
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_good'].values[0] 

        if sec_fin in ('D_G', 'Dtilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            f_d = empirical_d / common_d
            
            def solve_f_g(f_g):
                if f_g >= f_d:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    return empirical_g / common_g - ((f_g**((σ - xi) / σ) - f_d**((σ - xi) / σ)) / 
                                                     (f_g**(-xi / σ) - f_d**(-xi / σ)))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_d:
                initial_guess = f_d * 0.99  # Set initial guess to be less than f_d if necessary

            f_gg = fsolve(solve_f_g, initial_guess)[0]
    
    return f_gg, f_d

def calcul_fc_CG_good(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = 0
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_good'].values[0] 

        if sec_fin in ('Ctilde_Gtilde', 'C_G'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            f_c = empirical_c / common_c
            
            def solve_f_g(f_g):
                if f_g >= f_c:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    numerator = (f_g**((σ - xi) / σ)) * (Rtilde_d**(((σ - xi) * (σ - 1)) / σ)) - \
                                (f_c**((σ - xi) / σ)) * (Rtilde_c**(((σ - xi) * (σ - 1)) / σ))
                    denominator = (f_g**(-xi / σ)) * (Rtilde_d**((-xi * (σ - 1)) / σ)) - \
                                  (f_c**(-xi / σ)) * (Rtilde_c**((-xi * (σ - 1)) / σ))
                    return empirical_g / common_g - (numerator / denominator)/(Rtilde_d**(σ-1))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_c:
                initial_guess = f_c * 0.99  # Set initial guess to be less than f_d if necessary

            f_g = fsolve(solve_f_g, initial_guess)[0]             
            
    return f_g, f_c 

def calcul_fc_DCG_good(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = 0
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_good'].values[0] 
        
        if sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            diff_d_c = (empirical_d / common_d) / (Rtilde_c**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))
            
            def solve_f_c(f_c):
                if diff_d_c+f_c < ((Rtilde_c/Rtilde_d)**(σ-1))*f_c:
                    return 1e10
                else:
                    return (empirical_c / common_c - 
                            (f_c**((σ - xi) / σ) - 
                            (diff_d_c**((σ - xi) / σ)) * 
                            ((Rtilde_d**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))**((σ - xi) / σ))) / 
                            (f_c**(-xi / σ) - 
                            (diff_d_c**(-xi / σ)) * 
                            ((Rtilde_d**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))**(-xi / σ))))
            initial_guess = empirical_c / common_c
            if diff_d_c+initial_guess < ((Rtilde_c/Rtilde_d)**(σ-1))*initial_guess:
                initial_guess=diff_d_c*0.9/(((Rtilde_c/Rtilde_d)**(σ-1))-0.9)
            f_c = fsolve(solve_f_c, initial_guess)[0]
            f_d = diff_d_c + f_c
            
            def solve_f_g(f_g):
                if f_g >= f_c:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:

                    numerator = (f_g**((σ - xi) / σ)) * (Rtilde_d**(((σ - xi) * (σ - 1)) / σ)) - \
                                (f_c**((σ - xi) / σ)) * (Rtilde_c**(((σ - xi) * (σ - 1)) / σ))
                    denominator = (f_g**(-xi / σ)) * (Rtilde_d**((-xi * (σ - 1)) / σ)) - \
                                  (f_c**(-xi / σ)) * (Rtilde_c**((-xi * (σ - 1)) / σ))
                    return empirical_g / common_g - (numerator / denominator)/(Rtilde_d**(σ-1))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_c:
                initial_guess = f_c * 0.99  # Set initial guess to be less than f_d if necessary            
            f_g = fsolve(solve_f_g, initial_guess)[0]
    return f_g, f_d, f_c 

def calcul_fc_CDG_good(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = 0
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_good'].values[0] 
        
        if sec_fin in ('Ctilde_Dtilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            diff_c_d = (empirical_c / common_c) / (Rtilde_d**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))
            
            def solve_f_d(f_d):
                if f_d >= ((Rtilde_c/Rtilde_d)**(σ-1))*(f_d+diff_c_d):
                    return 1e10
                else:
                    return (empirical_d / common_d - 
                            (f_d**((σ - xi) / σ) - 
                            (diff_c_d**((σ - xi) / σ)) * 
                            ((Rtilde_c**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))**((σ - xi) / σ))) / 
                            (f_d**(-xi / σ) - 
                            (diff_c_d**(-xi / σ)) * 
                            ((Rtilde_c**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))**(-xi / σ))))
                
            initial_guess = empirical_d / common_d
            if initial_guess >= ((Rtilde_c/Rtilde_d)**(σ-1))*(initial_guess+diff_c_d):
                initial_guess = (((Rtilde_c/Rtilde_d)**(σ-1))*0.9*diff_c_d)/(1-0.9*((Rtilde_c/Rtilde_d)**(σ-1)))  # Set initial guess to be less than f_d if necessary                 
            f_d = fsolve(solve_f_d, initial_guess)[0]
            f_c = diff_c_d + f_d
            
            def solve_f_g(f_g):
                if f_g >= f_d:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    return empirical_g / common_g - ((f_g**((σ - xi) / σ) - f_d**((σ - xi) / σ)) / 
                                                     (f_g**(-xi / σ) - f_d**(-xi / σ)))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_d:
                initial_guess = f_d * 0.99  # Set initial guess to be less than f_d if necessary            
            f_g = fsolve(solve_f_g, initial_guess)[0]

    return f_g, f_d, f_c

In [8]:
def cal_fc_good(iso_code):
    query_result = data_prep2[data_prep2['iso_code'] == iso_code]
    sectors = np.sort(query_result['sector'].unique())
    fc_good = {}  # Use a dictionary for initial financial coefficients
    for sector in sectors:
        sector_data = query_result[query_result['sector'] == sector]
        sec_fin = sector_data['sec_fin_good'].iloc[0]
        
        # Dynamically create keys using the sector name
        if sec_fin in ('G', 'Gtilde'):
            fc_good[f'{sector}_f_g'] = calcul_fc_G_good(iso_code, sector)
            
        elif sec_fin in ('Dtilde_Gtilde', 'D_G'):
            fc_good[f'{sector}_f_g'], fc_good[f'{sector}_f_d'] = calcul_fc_DG_good(iso_code, sector)
            
        elif sec_fin in ('Ctilde_Gtilde', 'C_G'):
            fc_good[f'{sector}_f_g'], fc_good[f'{sector}_f_c'] = calcul_fc_CG_good(iso_code, sector)
            
        elif sec_fin in ('Dtilde_Ctilde_Gtilde', 'D_C_G', 'Dtilde_C_Gtilde'):
            fc_good[f'{sector}_f_g'], fc_good[f'{sector}_f_d'], fc_good[f'{sector}_f_c'] = calcul_fc_DCG_good(iso_code, sector)
            
        elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
            fc_good[f'{sector}_f_g'], fc_good[f'{sector}_f_d'], fc_good[f'{sector}_f_c'] = calcul_fc_CDG_good(iso_code, sector)
    
    return fc_good

In [79]:
cal_fc_good('RWA')

{'Agriculture, Forestry, Fishing_f_g': 20315.525856781493,
 'Agriculture, Forestry, Fishing_f_d': 178630.51974965716,
 'Communications_f_g': 4356.541380618283,
 'Communications_f_d': 606437.5658653813,
 'Education_f_g': 10238.39044806545,
 'Education_f_d': 436989.9520702092,
 'Energy_f_g': 76060.88121189504,
 'Energy_f_c': 12453565.484415954,
 'General Budget Support_f_g': 31936.791159546832,
 'General Budget Support_f_d': 666739.3926915888,
 'General Economic, Commercial, Labor Affairs_f_g': 21495.44815651839,
 'General Economic, Commercial, Labor Affairs_f_d': 212254.78149059138,
 'General Environment Protection_f_g': 14785.973668043032,
 'General Environment Protection_f_d': 105999.45903692402,
 'Government & Civil Society_f_g': 17319.893813396942,
 'Government & Civil Society_f_d': 247031.82928140115,
 'Health_f_g': 14550.908853469198,
 'Health_f_d': 189589.57726471624,
 'Industry, Mining, Construction_f_g': 5796.373061748301,
 'Industry, Mining, Construction_f_c': 5967350.57527593

In [42]:
query_result=data_prep2[data_prep2['iso_code'] == 'VNM']

In [44]:
sectors=np.sort(query_result['sector'].unique())

In [80]:
data_prep2[(data_prep2['iso_code'] == 'VNM')]['chi_bad'].values[0]

1.0934484

In [75]:
calcul_fc_DG_good('VNM','Communications')

(2682.7550937401825, 383863.8895468877)

In [9]:
def calcul_fc_G_bad(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = data_prep2[(data_prep2['iso_code'] == iso_code)]['chi_bad'].values[0]
    
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_bad'].values[0] 
        
        if sec_fin in ('G', 'Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            f_g = empirical_g / common_g
        
    return f_g 

def calcul_fc_DG_bad(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = data_prep2[(data_prep2['iso_code'] == iso_code)]['chi_bad'].values[0]
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_bad'].values[0] 

        if sec_fin in ('D_G', 'Dtilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            f_d = empirical_d / common_d
            
            def solve_f_g(f_g):
                if f_g >= f_d:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    return empirical_g / common_g - ((f_g**((σ - xi) / σ) - f_d**((σ - xi) / σ)) / 
                                                     (f_g**(-xi / σ) - f_d**(-xi / σ)))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_d:
                initial_guess = f_d * 0.99  # Set initial guess to be less than f_d if necessary

            f_gg = fsolve(solve_f_g, initial_guess)[0]
    
    return f_gg, f_d

def calcul_fc_CG_bad(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = data_prep2[(data_prep2['iso_code'] == iso_code)]['chi_bad'].values[0]
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_bad'].values[0] 

        if sec_fin in ('Ctilde_Gtilde', 'C_G'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            f_c = empirical_c / common_c
            
            def solve_f_g(f_g):
                if f_g >= f_c:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    numerator = (f_g**((σ - xi) / σ)) * (Rtilde_d**(((σ - xi) * (σ - 1)) / σ)) - \
                                (f_c**((σ - xi) / σ)) * (Rtilde_c**(((σ - xi) * (σ - 1)) / σ))
                    denominator = (f_g**(-xi / σ)) * (Rtilde_d**((-xi * (σ - 1)) / σ)) - \
                                  (f_c**(-xi / σ)) * (Rtilde_c**((-xi * (σ - 1)) / σ))
                    return empirical_g / common_g - (numerator / denominator)/(Rtilde_d**(σ-1))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_c:
                initial_guess = f_c * 0.99  # Set initial guess to be less than f_d if necessary

            f_g = fsolve(solve_f_g, initial_guess)[0]             
            
    return f_g, f_c 

def calcul_fc_DCG_bad(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = data_prep2[(data_prep2['iso_code'] == iso_code)]['chi_bad'].values[0]
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_bad'].values[0] 
        
        if sec_fin in ('D_C_G', 'Dtilde_C_Gtilde', 'Dtilde_Ctilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            diff_d_c = (empirical_d / common_d) / (Rtilde_c**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))
            
            def solve_f_c(f_c):
                if diff_d_c+f_c < ((Rtilde_c/Rtilde_d)**(σ-1))*f_c:
                    return 1e10
                else:
                    return (empirical_c / common_c - 
                            (f_c**((σ - xi) / σ) - 
                            (diff_d_c**((σ - xi) / σ)) * 
                            ((Rtilde_d**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))**((σ - xi) / σ))) / 
                            (f_c**(-xi / σ) - 
                            (diff_d_c**(-xi / σ)) * 
                            ((Rtilde_d**(σ - 1) / (Rtilde_c**(σ - 1) - Rtilde_d**(σ - 1)))**(-xi / σ))))
            initial_guess = empirical_c / common_c
            if diff_d_c+initial_guess < ((Rtilde_c/Rtilde_d)**(σ-1))*initial_guess:
                initial_guess=diff_d_c*0.9/(((Rtilde_c/Rtilde_d)**(σ-1))-0.9)
            f_c = fsolve(solve_f_c, initial_guess)[0]
            f_d = diff_d_c + f_c
            
            def solve_f_g(f_g):
                if f_g >= f_c:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:

                    numerator = (f_g**((σ - xi) / σ)) * (Rtilde_d**(((σ - xi) * (σ - 1)) / σ)) - \
                                (f_c**((σ - xi) / σ)) * (Rtilde_c**(((σ - xi) * (σ - 1)) / σ))
                    denominator = (f_g**(-xi / σ)) * (Rtilde_d**((-xi * (σ - 1)) / σ)) - \
                                  (f_c**(-xi / σ)) * (Rtilde_c**((-xi * (σ - 1)) / σ))
                    return empirical_g / common_g - (numerator / denominator)/(Rtilde_d**(σ-1))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_c:
                initial_guess = f_c * 0.99  # Set initial guess to be less than f_d if necessary            
            f_g = fsolve(solve_f_g, initial_guess)[0]
    return f_g, f_d, f_c 

def calcul_fc_CDG_bad(iso_code, sector):
    xi = xi_vec[iso_code]
    psi_d = psi_vec[sector]['DAC']
    psi_c = psi_vec[sector]['China']
    R_d = R_vec[sector]['DAC']
    R_c = R_vec[sector]['China']
    chi = data_prep2[(data_prep2['iso_code'] == iso_code)]['chi_bad'].values[0]
    Rtilde_d = (R_d - (1 - psi_d) * chi) / psi_d - (1 - δ)
    Rtilde_c = (R_c - (1 - psi_c) * chi) / psi_c - (1 - δ)
    
    common_d = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_g = xi * (σ - 1) / (psi_d * Rtilde_d * (xi - σ))
    common_c = xi * (σ - 1) / (psi_c * Rtilde_c * (xi - σ))
    
    query_result = data_prep2[(data_prep2['iso_code'] == iso_code) & (data_prep2['sector'] == sector)]

    if not query_result.empty:
        sec_fin = query_result['sec_fin_bad'].values[0] 
        
        if sec_fin in ('Ctilde_Dtilde_Gtilde'):
            empirical_g = query_result['size_dacgrant'].values[0]
            empirical_d = query_result['size_dacloan'].values[0]
            empirical_c = query_result['size_chinaloan'].values[0]
            diff_c_d = (empirical_c / common_c) / (Rtilde_d**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))
            
            def solve_f_d(f_d):
                if f_d >= ((Rtilde_c/Rtilde_d)**(σ-1))*(f_d+diff_c_d):
                    return 1e10
                else:
                    return (empirical_d / common_d - 
                            (f_d**((σ - xi) / σ) - 
                            (diff_c_d**((σ - xi) / σ)) * 
                            ((Rtilde_c**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))**((σ - xi) / σ))) / 
                            (f_d**(-xi / σ) - 
                            (diff_c_d**(-xi / σ)) * 
                            ((Rtilde_c**(σ - 1) / (Rtilde_d**(σ - 1) - Rtilde_c**(σ - 1)))**(-xi / σ))))
                
            initial_guess = empirical_d / common_d
            if initial_guess >= ((Rtilde_c/Rtilde_d)**(σ-1))*(initial_guess+diff_c_d):
                initial_guess = (((Rtilde_c/Rtilde_d)**(σ-1))*0.9*diff_c_d)/(1-0.9*((Rtilde_c/Rtilde_d)**(σ-1)))  # Set initial guess to be less than f_d if necessary                 
            f_d = fsolve(solve_f_d, initial_guess)[0]
            f_c = diff_c_d + f_d
            
            def solve_f_g(f_g):
                if f_g >= f_d:  # Check if f_g is greater than or equal to f_d
                    return 1e10  # Return a large number to indicate a bad solution
                else:
                    return empirical_g / common_g - ((f_g**((σ - xi) / σ) - f_d**((σ - xi) / σ)) / 
                                                     (f_g**(-xi / σ) - f_d**(-xi / σ)))

            initial_guess = empirical_g / common_g
            if initial_guess >= f_d:
                initial_guess = f_d * 0.6  # Set initial guess to be less than f_d if necessary            
            f_g = fsolve(solve_f_g, initial_guess)[0]

    return f_g, f_d, f_c

In [10]:
def cal_fc_bad(iso_code):
    query_result = data_prep2[data_prep2['iso_code'] == iso_code]
    sectors = np.sort(query_result['sector'].unique())
    fc_bad = {}  # Use a dictionary for initial financial coefficients
    for sector in sectors:
        sector_data = query_result[query_result['sector'] == sector]
        sec_fin = sector_data['sec_fin_bad'].iloc[0]
        
        # Dynamically create keys using the sector name
        if sec_fin in ('G', 'Gtilde'):
            fc_bad[f'{sector}_f_g'] = calcul_fc_G_bad(iso_code, sector)
            
        elif sec_fin in ('Dtilde_Gtilde', 'D_G'):
            fc_bad[f'{sector}_f_g'], fc_bad[f'{sector}_f_d'] = calcul_fc_DG_bad(iso_code, sector)
            
        elif sec_fin in ('Ctilde_Gtilde', 'C_G'):
            fc_bad[f'{sector}_f_g'], fc_bad[f'{sector}_f_c'] = calcul_fc_CG_bad(iso_code, sector)
            
        elif sec_fin in ('Dtilde_Ctilde_Gtilde', 'D_C_G', 'Dtilde_C_Gtilde'):
            fc_bad[f'{sector}_f_g'], fc_bad[f'{sector}_f_d'], fc_bad[f'{sector}_f_c'] = calcul_fc_DCG_bad(iso_code, sector)
            
        elif sec_fin in ('Ctilde_Dtilde_Gtilde'):
            fc_bad[f'{sector}_f_g'], fc_bad[f'{sector}_f_d'], fc_bad[f'{sector}_f_c'] = calcul_fc_CDG_bad(iso_code, sector)
    
    return fc_bad

C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


{'Agriculture, Forestry, Fishing_f_g': 7654.740977624708,
 'Agriculture, Forestry, Fishing_f_c': 1715059.51137572,
 'Communications_f_g': 2682.7550937401825,
 'Communications_f_d': 383863.8895468877,
 'Education_f_g': 8708.527333650944,
 'Education_f_d': 581673.6065885938,
 'Energy_f_g': 20680.836957974985,
 'Energy_f_c': 5416894.477155423,
 'General Budget Support_f_g': 20552.229824354392,
 'General Budget Support_f_d': 439835.8316657741,
 'General Budget Support_f_c': 256360.13926960158,
 'General Economic, Commercial, Labor Affairs_f_g': 9700.862021288212,
 'General Economic, Commercial, Labor Affairs_f_c': 748730.3797025123,
 'General Environment Protection_f_g': 12292.832498007587,
 'General Environment Protection_f_d': 719773.8484465352,
 'Government & Civil Society_f_g': 6882.75394549079,
 'Government & Civil Society_f_d': 601221.5117116242,
 'Health_f_g': 11722.013616723298,
 'Health_f_d': 336953.51244189654,
 'Industry, Mining, Construction_f_g': 5268.214887367363,
 'Industry,

In [39]:
fixed_costs_estimates_bad = []
fixed_costs_estimates_good = []

In [40]:
for iso_code in data_prep2['iso_code'].unique() :
    results = cal_fc_bad(iso_code)
    for key, value in results.items():
        # Split the key by underscore and handle the combination of sector and cost type
        parts = key.rsplit('_', 2)  # Split from the right, expecting two splits at most
        sector = parts[0]  # Everything before the last underscore is the sector
        cost_type = parts[1] + '_' + parts[2]  # Combine the last two parts for the cost type
        fixed_costs_estimates_bad.append({'iso_code': iso_code, 'sector': sector, 'cost_type': cost_type, 'estimate': value})

# Convert the list to a DataFrame
df_bad = pd.DataFrame(fixed_costs_estimates_bad)

# Show the DataFrame
print(df_bad.head())

# If you want to save this to a CSV file
df_bad.to_csv('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\estimated_fixed_costs_bad.csv', index=False)

C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


  iso_code                          sector cost_type      estimate
0      AGO  Agriculture, Forestry, Fishing       f_g   8334.952724
1      AGO  Agriculture, Forestry, Fishing       f_c  46395.938427
2      AGO                  Communications       f_g  11463.254756
3      AGO                  Communications       f_d  11463.588753
4      AGO                  Communications       f_c  19172.026003


In [41]:
for iso_code in data_prep2['iso_code'].unique() :
    results = cal_fc_good(iso_code)
    for key, value in results.items():
        # Split the key by underscore and handle the combination of sector and cost type
        parts = key.rsplit('_', 2)  # Split from the right, expecting two splits at most
        sector = parts[0]  # Everything before the last underscore is the sector
        cost_type = parts[1] + '_' + parts[2]  # Combine the last two parts for the cost type
        fixed_costs_estimates_good.append({'iso_code': iso_code, 'sector': sector, 'cost_type': cost_type, 'estimate': value})

# Convert the list to a DataFrame
df_good = pd.DataFrame(fixed_costs_estimates_good)

# Show the DataFrame
print(df_good.head())

# If you want to save this to a CSV file
df_good.to_csv('C:\\Users\\chans\\Desktop\\UpdatedEmpirics\\estimated_fixed_costs_good.csv', index=False)

C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\chans\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


  iso_code                          sector cost_type       estimate
0      AGO  Agriculture, Forestry, Fishing       f_g    2925.921346
1      AGO  Agriculture, Forestry, Fishing       f_c  216791.577127
2      AGO                  Communications       f_g    2467.860480
3      AGO                  Communications       f_c  447069.747036
4      AGO                       Education       f_g    7397.081752
